In [1]:
# As usual, a bit of setup
import numpy as np
import matplotlib.pyplot as plt
from cs231n.classifiers.cnn import *
from cs231n.data_utils import get_CIFAR10_data

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [2]:
data = get_CIFAR10_data()
for k, v in data.items():
  print('%s: ' % k, v.shape)

X_train:  (49000, 3, 32, 32)
y_train:  (49000,)
X_val:  (1000, 3, 32, 32)
y_val:  (1000,)
X_test:  (1000, 3, 32, 32)
y_test:  (1000,)


In [3]:
X_train = np.moveaxis(data['X_train'], 1, 3)
X_val = np.moveaxis(data['X_val'], 1, 3)
X_test = np.moveaxis(data['X_test'], 1, 3)
print(X_train.shape, X_val.shape, X_test.shape)

(49000, 32, 32, 3) (1000, 32, 32, 3) (1000, 32, 32, 3)


In [4]:
mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
X_train = (X_train - mean_pixel) / std_pixel
X_val = (X_val - mean_pixel) / std_pixel
X_test = (X_test - mean_pixel) / std_pixel

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Convolution2D, Flatten, Activation, Input, MaxPooling2D, Dropout, Conv2D
from keras.initializers import VarianceScaling, Zeros
from keras import Model
from keras.utils import np_utils
import tensorflow as tf

In [9]:
y_train = np_utils.to_categorical(data['y_train'], 10)
y_val = np_utils.to_categorical(data['y_val'], 10)
y_test = np_utils.to_categorical(data['y_test'], 10)

In [18]:
initializer = VarianceScaling(scale=2.0)
zero_initializer = Zeros()
model = Sequential([
    Conv2D(
        filters=32,
        kernel_size=5,
        strides=1,
        padding='same',
        data_format='channels_first',
        activation='relu',
        use_bias=True,
        kernel_initializer = initializer,
        bias_initializer = zero_initializer
        
        
    ),
    MaxPooling2D(
        pool_size=(2, 2)
    ),
    Dropout(
        rate=0.2
    ),
    Conv2D(
        filters=16,
        kernel_size=3,
        strides=1,
        padding='same',
        data_format='channels_first',
        activation='relu',
        use_bias=True,
        kernel_initializer = initializer,
        bias_initializer = zero_initializer
        
        
    ),
    Flatten(
    data_format='channels_first'
    ),
    Dense(
        units=10,
        use_bias=True,
        kernel_initializer = initializer,
        bias_initializer = zero_initializer,
        activation='softmax'
    )
    
])

In [19]:
optimaizer  = tf.train.MomentumOptimizer(0.01, 0.9)
model.compile(optimizer=optimaizer,
              loss='mean_squared_error',
              metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train,                # Train the model using the training set...
          batch_size=32, epochs=5,
          verbose=1, validation_data=(X_val, y_val)) # ...holding out 10% of the data for validation

Train on 49000 samples, validate on 1000 samples
Epoch 1/5
49000/49000 [==============================] - 52s 1ms/step - loss: 0.0679 - acc: 0.4633 - val_loss: 0.0660 - val_acc: 0.4940
Epoch 2/5
49000/49000 [==============================] - 56s 1ms/step - loss: 0.0663 - acc: 0.4789 - val_loss: 0.0653 - val_acc: 0.4900
Epoch 3/5
49000/49000 [==============================] - 57s 1ms/step - loss: 0.0651 - acc: 0.4899 - val_loss: 0.0632 - val_acc: 0.5120
Epoch 4/5
49000/49000 [==============================] - 47s 956us/step - loss: 0.0641 - acc: 0.4996 - val_loss: 0.0629 - val_acc: 0.5160
Epoch 5/5
49000/49000 [==============================] - 48s 970us/step - loss: 0.0628 - acc: 0.5137 - val_loss: 0.0609 - val_acc: 0.5240


In [23]:
model.evaluate(X_test, y_test)

1000/1000 [==============================] - 0s 492us/step


[0.061386766225099566, 0.52600000000000002]

In [30]:
y_predict = model.predict(X_test)
print(y_predict.shape)
y_predict = np.argmax(y_predict, axis=1)
y_true = np.argmax(y_test, axis=1)
np.count_nonzero(y_true == y_predict)/ y_predict.shape[0]

(1000, 10)


0.526

In [7]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [ ]:
X_train = np.s

In [8]:
num_train, height, width, depth = X_train.shape # there are 50000 training examples in CIFAR-10 
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = 10 # there are 10 image classe

In [13]:
inp = Input(shape=(height, width, depth)) # depth goes last in TensorFlow back-end (first in Theano)
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

In [16]:
model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

In [17]:
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

In [24]:
model.fit(X_train, y_train,                # Train the model using the training set...
          batch_size=batch_size, epochs=2,
          verbose=1, validation_data=(X_val, y_val)) # ...holding out 10% of the data for validation

Train on 49000 samples, validate on 1000 samples
Epoch 1/2
35968/49000 [=====================>........] - ETA: 1:10 - loss: 14.5040 - acc: 0.1001

KeyboardInterrupt: 